<a href="https://colab.research.google.com/github/manavgurnani21/data_augmentation_tld_research/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adding Images to Drive Folder

In [ ]:
!pip install -q kaggle

from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mbornoe/lisa-traffic-light-dataset
!unzip lisa-traffic-light-dataset.zip

In [ ]:
import os

# getting current directory
os.getcwd()

all_image_paths = []

## Getting Day Sequence Paths

In [ ]:
# getting all paths for content layer
content = os.listdir('/content/')
content.sort()
content = content[:-3]
content.remove('.config')
content.remove('kaggle.json')
content.remove('lisa-traffic-light-dataset.zip')
content.remove('Annotations')
content.remove('dayTrain')
content.remove('nightTrain')
for folder in content:
  if folder == '.ipynb_checkpoints':
    content.remove('.ipynb_checkpoints')
print(content)

['content', 'daySequence1', 'daySequence2', 'drive', 'nightSequence1', 'nightSequence2', 'sample-dayClip6', 'sample-nightClip1', 'sample_data']


In [ ]:
for folder in content:
  print('/content/' + folder + '/' + folder + '/frames/')
  list = os.listdir('/content/' + folder + '/' + folder + '/frames/')
  for path in list:
    all_image_paths.append('/content/' + folder + '/' + folder + '/frames/' + path)

/content/content/content/frames/


FileNotFoundError: ignored

## Getting Clip Paths

In [ ]:
train_paths = ['/content/dayTrain/dayTrain/', '/content/nightTrain/nightTrain/']
for path in train_paths:
  list1 = os.listdir(path)
  if '.DS_Store' in list1:
    list1.remove('.DS_Store')
  for name in list1:
    list2 = os.listdir(path + name + '/frames/')
    for item in list2:
      all_image_paths.append(path + name + '/frames/' + item)

# Adding All Annotations

## Getting all Sequence Annotations

In [ ]:
import os

all_annotation_paths = []

In [ ]:
root_path = '/content/Annotations/Annotations'
main = os.listdir(root_path)
main.remove('dayTrain')
main.remove('nightTrain')

for folder in main:
  list1 = os.listdir(root_path + '/' + folder)
  list1[0] = folder + list1[0]
  os.rename(root_path + folder + '/frameAnnotationsBOX.csv', root_path + folder + '/' + list1[0])
  all_annotation_paths.append(root_path + folder + '/' + list1[0])

## Getting all Clip Annotations

In [ ]:
clipPaths = [root_path + 'dayTrain/', root_path + 'nightTrain/']

for folder in clipPaths:
  list2 = os.listdir(folder)
  for name in list2:
    list3 = os.listdir(folder + name)
    list3[0] = name + list3[0]
    print(folder + name + '/' + list3[0])
    os.rename(folder + name + '/frameAnnotationsBOX.csv', folder + name + '/' + list3[0])
    all_annotation_paths.append(folder + name + '/' + list3[0])

# Sorting All Lists

In [ ]:
import numpy as np
image_paths = np.asarray(all_image_paths)
sorted_image_paths = np.sort(image_paths)
print(sorted_image_paths)

annotation_paths = np.asarray(all_annotation_paths)
sorted_annotation_paths = np.sort(annotation_paths)
print(sorted_annotation_paths)

# Cropping the Images

In [ ]:
def findIndexofElement(value, array):
  for i in range(len(array)):
    if array[i][array[i].rfind('/'):] == value:
      return i
      break

In [ ]:
os.mkdir('/content/allCroppedImages/')
os.mkdir('/content/allCroppedImages/stop/')
os.mkdir('/content/allCroppedImages/warning/')
os.mkdir('/content/allCroppedImages/go/')
os.mkdir('/content/allCroppedImages/warningLeft/')
os.mkdir('/content/allCroppedImages/goLeft/')
os.mkdir('/content/allCroppedImages/stopLeft/')
os.mkdir('/content/allCroppedImages/goForward/')

In [ ]:
from google.colab.patches import cv2_imshow
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.utils import load_img
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import os

def cropAllImages(path):
  df = pd.read_csv(path, sep=';')
  filenames = df['Filename']
  leftX = np.asarray(df['Upper left corner X'])
  rightX = np.asarray(df['Lower right corner X'])
  leftY = np.asarray(df['Upper left corner Y'])
  rightY = np.asarray(df['Lower right corner Y'])
  tag = np.asarray(df['Annotation tag'])

  image_saved_counter = 0

  # loc_index is the location of the image path in all sorted paths
  for i in range(len(filenames)):
    findIndexofElement(filenames[i][filenames[i].rfind('/'):], sorted_image_paths)
    img = img_to_array(load_img(sorted_image_paths[findIndexofElement(filenames[i][filenames[i].rfind('/'):], sorted_image_paths)]))
    crop_img = array_to_img(img[leftY[i]:rightY[i], leftX[i]:rightX[i]])
    # inputting them in folder
    crop_img.save('/content/allCroppedImages/' + tag[i] + filenames[i][filenames[i].rfind('/'):])
    image_saved_counter+=1
    if(image_saved_counter%1000==0):
      print(image_saved_counter)

# for path in all_annotation_paths:
#   cropAllImages(path)

In [ ]:
# from google.colab import files
# !zip -r '/content/"allCroppedImages.zip"' '/content/allCroppedImages'
# files.download('/content/allCroppedImages.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Randomly Assigning Files

- after putting into sub-folders
- for each subfolder:
  - put all names in a list
  - shuffle
  - get all three indices
  - put into train, test, val

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip '/content/drive/MyDrive/allCroppedImages/allCroppedImages.zip' -d '/content/'

In [ ]:
import os

os.mkdir('/content/train/')
os.mkdir('/content/test/')
os.mkdir('/content/val/')

folderList = ['train', 'test', 'val']
for name in folderList:
  os.mkdir('/content/' + name + '/stop/')
  os.mkdir('/content/' + name + '/go/')
  os.mkdir('/content/' + name + '/warning/')
  os.mkdir('/content/' + name + '/warningLeft/')
  os.mkdir('/content/' + name + '/goLeft/')
  os.mkdir('/content/' + name + '/stopLeft/')
  os.mkdir('/content/' + name + '/goForward/')

In [ ]:
import random

def listPaths(path):
  pathList = []
  for folder in os.listdir(path):
    if folder == '.ipynb_checkpoints':
      continue
    pathList.append(path + folder + '/')
    random.shuffle(pathList)
  return pathList

print(listPaths('/content/content/allCroppedImages/'))

['/content/content/allCroppedImages/warning/', '/content/content/allCroppedImages/stop/', '/content/content/allCroppedImages/goForward/', '/content/content/allCroppedImages/go/', '/content/content/allCroppedImages/goLeft/', '/content/content/allCroppedImages/warningLeft/', '/content/content/allCroppedImages/stopLeft/']


In [ ]:
import random
import shutil

def shuffleSelection(path):
  allFolders = listPaths(path)
  for folder in allFolders:
    df = pd.DataFrame(listPaths(folder))
    trainPaths, testPaths, valPaths = np.split(df, [int(.8 * len(df)), int(.9 * len(df))])
    moveToFolder(trainPaths, testPaths, valPaths)

def moveToFolder(trainPaths, testPaths, valPaths):
  finalTrainPathList = np.asarray(trainPaths[0])
  type(finalTrainPathList)
  finalTestPathList = np.asarray(testPaths[0])
  finalValPathList = np.asarray(valPaths[0])
  for path in finalTrainPathList:
    shutil.move(path[:-1], '/content/train' + path[33:-1])
  for path in finalTestPathList:
    shutil.move(path[:-1], '/content/test' + path[33:-1])
  for path in finalValPathList:
    shutil.move(path[:-1], '/content/val' + path[33:-1])

In [ ]:
shuffleSelection('/content/content/allCroppedImages/')

In [ ]:
import os, shutil
folder = '/content/sample-nightClip1/'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

# Training Dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

train_data_dir = '/content/train/'
test_data_dir = '/content/test/'
val_data_dir = '/content/val/'
 
img_height = 180
img_width = 180
batch_size=32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  val_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 40757 files belonging to 7 classes.
Found 5096 files belonging to 7 classes.
Found 5098 files belonging to 7 classes.


In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(180,180,3),
                   pooling='avg',classes=5,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(5, activation='softmax'))

In [ ]:
resnet_model.summary()

In [ ]:
resnet_model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("best_model_ever.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
history = resnet_model.fit(
  train_ds,
  validation_data=val_ds,
  callbacks=[early, checkpoint],
  epochs=10
)

Epoch 1/10
   1/1274 [..............................] - ETA: 1:00:51 - loss: 1.2707 - accuracy: 0.7500

InvalidArgumentError: ignored

## Agenda for 10/18

- sort out issue with random shuffle function (ask about cropping time)
- find way to convert images to dataset
  - ask why we need singular class folders

Goals for the next two weeks:
- run experiments (and caputre results)
- finish research paper